In [7]:
import pandas as pd
import numpy as np

In [8]:
leaks = pd.read_csv('./Data/NewData/Leaks.csv')
leaks.head(10)

,Date,Kandy,Dambulla,Buy Rate
0,2020-01-01,75.000000,305.00,179.4845
1,2020-01-02,75.000000,265.00,179.2450
2,2020-01-03,75.000000,225.00,179.3145
3,2020-01-04,75.000000,185.00,179.3346
4,2020-01-05,75.000000,145.00,179.3547
5,2020-01-06,75.000000,105.00,179.3748
6,2020-01-07,75.000000,65.00,179.3846
7,2020-01-08,68.333333,60.00,179.3748
8,2020-01-09,61.666667,55.00,179.6836
9,2020-01-10,123.750000,121.25,179.6164


In [9]:
leaks = leaks.round(2)
leaks.head(10)

,Date,Kandy,Dambulla,Buy Rate
0,2020-01-01,75.00,305.00,179.48
1,2020-01-02,75.00,265.00,179.24
2,2020-01-03,75.00,225.00,179.31
3,2020-01-04,75.00,185.00,179.33
4,2020-01-05,75.00,145.00,179.35
5,2020-01-06,75.00,105.00,179.37
6,2020-01-07,75.00,65.00,179.38
7,2020-01-08,68.33,60.00,179.37
8,2020-01-09,61.67,55.00,179.68
9,2020-01-10,123.75,121.25,179.62


In [12]:
leaks['Buy Rate Lagged'] = leaks['Buy Rate'].shift(7)
leaks.head(10)

,Date,Kandy,Dambulla,Buy Rate,Buy Rate Lagged
0,2020-01-01,75.00,305.00,179.48,NaN
1,2020-01-02,75.00,265.00,179.24,NaN
2,2020-01-03,75.00,225.00,179.31,NaN
3,2020-01-04,75.00,185.00,179.33,NaN
4,2020-01-05,75.00,145.00,179.35,NaN
5,2020-01-06,75.00,105.00,179.37,NaN
6,2020-01-07,75.00,65.00,179.38,NaN
7,2020-01-08,68.33,60.00,179.37,179.48
8,2020-01-09,61.67,55.00,179.68,179.24
9,2020-01-10,123.75,121.25,179.62,179.31


In [13]:
leaks = leaks.fillna(method='bfill')

C:\Users\HP\AppData\Local\Temp\ipykernel_2700\16135228.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  leaks = leaks.fillna(method='bfill')


In [17]:
leaks.tail(10)

,Date,Kandy,Dambulla,Buy Rate,Buy Rate Lagged
1449,2023-12-20,240.0,230.0,321.89,321.77
1450,2023-12-21,270.0,265.0,321.92,322.24
1451,2023-12-22,260.0,240.0,321.46,321.69
1452,2023-12-23,265.0,254.0,321.28,321.78
1453,2023-12-24,270.0,268.0,321.09,321.86
1454,2023-12-25,275.0,282.0,320.91,321.95
1455,2023-12-26,280.0,296.0,320.72,321.93
1456,2023-12-27,285.0,310.0,320.53,321.89
1457,2023-12-28,295.0,290.0,319.15,321.92
1458,2023-12-29,395.0,515.0,319.18,321.46


In [15]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(leaks[['Kandy', 'Dambulla', 'Buy Rate Lagged']])

# Function to create sequences for LSTM
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])  # Input sequence
        y.append(data[i+seq_length][:2])  # Target sequence (next step)
    return np.array(X), np.array(y)

# Define sequence length
seq_length = 7

# Create sequences
X, y = create_sequences(scaled_data, seq_length)

# # Split the data into training, validation, and test sets
# train_ratio = 0.7
# val_ratio = 0.15
# test_ratio = 0.15

# train_index = int(len(X) * train_ratio)
# val_index = int(len(X) * (train_ratio + val_ratio))

# X_train, X_val, X_test = X[:train_index], X[train_index:val_index], X[val_index:]
# y_train, y_val, y_test = y[:train_index], y[train_index:val_index], y[val_index:]

# # Display the shape of the resulting arrays
# print("Training Set:", X_train.shape, y_train.shape)
# print("Validation Set:", X_val.shape, y_val.shape)
# print("Test Set:", X_test.shape, y_test.shape)

In [24]:
future_buy_rate_lagged = leaks['Buy Rate'].tail(7).values.reshape(-1, 1)
scaler = MinMaxScaler()
scaled_future_buy_rate_lagged = scaler.fit_transform(future_buy_rate_lagged)
print(scaled_future_buy_rate_lagged)

[[1.        ]
 [0.91079812]
 [0.82629108]
 [0.7370892 ]
 [0.64788732]
 [0.        ]
 [0.01408451]]


In [26]:
import pickle
import tensorflow as tf
from tensorflow.keras import models

pickle_path = './Models/Leaks.pkl'

with open(pickle_path, 'rb') as file:
    model = pickle.load(file)

In [28]:
n_features = 3

last_seq = np.array(X[-1])
last_seq = last_seq.reshape(1, seq_length, n_features)

future_predictions = []

for i in range(7):
    next_step = model.predict(last_seq)
    future_predictions.append(next_step[0])
    new_input = np.array([[next_step[0][0], next_step[0][1], scaled_future_buy_rate_lagged[i][0]]])
    last_seq = np.append(last_seq[:, 1:, :], new_input.reshape(1, 1, n_features), axis=1)

future_predictions = np.array(future_predictions)

print(future_predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 784ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
[[0.61283505 0.44168785]
 [0.54933757 0.4006224 ]
 [0.50988144 0.3753146 ]
 [0.4968363  0.3673152 ]
 [0.50643253 0.37397963]
 [0.52807015 0.38812938]
 [0.5977932  0.43246844]]


In [29]:
predictions_resclaed = scaler.inverse_transform(np.concatenate([future_predictions, np.zeros((future_predictions.shape[0], 1))], axis=1)
)[:, :2]
print(predictions_resclaed)

[[320.45533866 320.09079513]
 [320.32008902 320.00332571]
 [320.23604746 319.94942008]
 [320.20826133 319.93238138]
 [320.2287013  319.94657661]
 [320.27478942 319.97671559]
 [320.42329956 320.07115779]]
